# Sturddle: A Chess Engine and Application

## Introduction
Sturddle is developed as a tool to explore and learn from the fields of Good Old-Fashioned Artificial Intelligence (GOFAI) and modern AI. The project uses chess, a domain with rich resources such as extensive literature, numerous existing engine implementations, and a vast array of training data, making it an effective medium for studying AI.

## Technical Composition
The engine is a blend of C++, Cython, and Python, integrating various search algorithms including MTD(f), Negascout, and Negamax, with MTD(f) as the default choice for its balance of efficiency and search depth. See: https://en.wikipedia.org/wiki/MTD(f)

## Evolution of Evaluation Methods in Chess Engines

### Traditional Evaluation and the Integration of NNUE in Stockfish
- **Traditional Methods**: Chess engines like Stockfish traditionally used heuristics including material balance, pawn structure, piece activity, and king safety.
- **NNUE in Stockfish**: In August 2020, Stockfish 12 integrated the Efficiently Updatable Neural Network (NNUE), marking a significant evolution in chess engine evaluation methods. By 2021, leading chess engines, including Komodo Dragon, adopted NNUE for enhanced positional understanding without the need for GPU resources.

### Evaluation Approach in Sturddle
- **Neural Network (NN) Evaluation**: Sturddle uses a neural network for position evaluation when the material balance is within a specific margin. This method is effective for games with complex positional nuances.
- **Material Delta Evaluation**: For positions with a material imbalance exceeding a defined threshold, Sturddle employs a simpler material delta evaluation. This fast approach is suitable for positions where the material difference significantly influences the outcome.

This dual evaluation strategy in Sturddle optimizes efficiency and effectiveness across various chess positions, blending nuanced neural network analysis with straightforward material evaluation.

## Neural Network Integration
In its first iteration, Sturddle utilized the [NNUE Probe Library](https://github.com/dshawul/nnue-probe), leveraging NNUE models developed by the Stockfish Community. This version achieved an ELO rating of around 3070, as evaluated by the Computer Chess Rating Lists (CCRL).

The second version of Sturddle is enhanced with a neural network designed from the ground up and trained via a custom Tensorflow script. This version has shown improved performance in initial tests, though it has not yet been submitted for formal evaluation by the CCRL.

## Project Focus
Sturddle is more than just a chess engine; it is a platform for experimenting with and understanding AI. The project uses chess as a practical tool to investigate both historical and modern approaches in AI, serving as a valuable learning and research medium in the field.

## Neural Network Training and Inference

### Training Data Sources
The neural network is trained on a diverse dataset encompassing games from Lichess, Leela Chess Zero, and games played by my own engine. Incorporating data from the engine itself serves as a form of reinforced learning, enhancing the adaptability and depth of the learning process.

### Training Environment and Dataset
Training is conducted on a Linux machine equipped with an RTX 3090 graphics card. The datasets used are:
- A smaller dataset consisting of 6.5 billion moves.
- A larger dataset, currently at 14.2 billion moves, with a target of reaching 20 billion by the end of the year.

Data is stored in HD5 file format, optimizing the storage and retrieval processes.

### Feature Representation
The engine uses occupancy bitmaps for chess board representation, detailing 6 piece types for each color across 64 squares. This design facilitates the conversion of these bitmaps into neural network input features. The input layer comprises:
- 768 features for the occupancy bitmaps.
- 1 feature for the side to move.
- 128 additional features, derived by overparameterizing the occupancy masks for black and white, totaling 769 features.

### Loss Function and Data Filtering
- **Huber Loss Variation**: The training of the neural network utilizes a variation of the Huber loss function. This choice is due to its robustness in handling outliers, which is particularly useful in the diverse set of chess positions and evaluations.
- **Evaluation Range Filtering**: The training script includes a feature to filter out evaluations (Y values) beyond a certain range. By default, only evaluations within the range of [-15000, 15000] centipawns are used. This filtering helps focus the training on more typical game scenarios and avoids extreme cases that are less relevant for general gameplay

Details on the Loss Function design can be found [here](https://github.com/cristivlas/sturddle-2/blob/master/softclip.ipynb).

### Experimentation with Training Parameters
- **Filter and Delta Values**: Ongoing experiments are being conducted to find optimal values for both the filter threshold and the delta parameter of the Huber loss function. For instance, one effective version of the network was trained with a delta of 3.0 and a filter threshold of 1500 centipawns.
- **Current Training Runs**: Training runs with varying filter and delta values are underway to evaluate their impact on the network's performance and learning efficiency.

### Network Fine-Tuning
- **Selective Freezing of Network Layers**: Modifications have been made to the training script to allow parts of the neural network to be "frozen." This technique enables fine-tuning of the network, where specific layers are fixed, and learning is focused on other parts of the network. This approach is beneficial for refining the network's performance based on previous learning while adapting to new data or strategies.

These enhancements and experiments reflect a continuous effort to optimize the neural network's performance for chess analysis and decision-making. The ongoing training and adjustments are aimed at achieving a balance between accuracy, generalization, and computational efficiency.

### Inference
In line with techniques used in NNUE (Neural Networks Updated Efficiently), our inference process takes advantage of the binary nature of input features (0 or 1) and the minimal changes introduced by chess moves. This allows for the incremental computation of the first layer of the neural network, rather than full dot products for each move. We maintain a stack of states during the alpha-beta graph search, with individual stacks for each core to avoid race conditions.

### CPU-Only Inference for Wide Compatibility
Inference is implemented in C++ for CPU-only execution, ensuring compatibility with low-end Android devices. The implementation makes use of:
- Agner Fog's VCL2 ([GitHub Repository](https://github.com/vectorclass/version2))
- simde ([GitHub Repository](https://github.com/simd-everywhere/simde))
- Custom NEON code to optimize performance with `__ARM_FEATURE_FP16_VECTOR_ARITHMETIC` where available.

## Other Machine Learning Tools and Methodologies

### Optimization Tools
Several machine learning tools are instrumental in the development of the engine:
- A script based on Nevergrad is used for optimizing engine parameters. [Lakas](https://github.com/fsmosca/Lakas)
- A Bayesian tuner is employed for fine-tuning one or two-dimensional parameters. [Chess Tuning Tools Documentation](https://chess-tuning-tools.readthedocs.io/en/latest/).

### UCI Integration and Tuning Mode
When compiled in tuning mode, the engine exposes its internal parameters to the Universal Chess Interface (UCI). In the release mode, these parameters are set as compile-time constants. This dual-mode functionality allows for extensive testing and optimization during development.

### Data Collection for Reinforced Learning
During tuning runs, there is an option to enable data collection. Evaluations from the neural network, particularly from the winning perspective, are collected and fed back into the training process. This approach reinforces the learning capabilities of the neural network.

## Training Methodology and Architecture

### Experimentation with Perspective-Based Training
We have experimented with training the neural network on values from the perspective of the side-to-move, yielding reasonable results. The current architecture in development, however, is trained on ground truth evaluations from the white player's perspective. These evaluations are switched post-inference at runtime.

### Network Architecture
The neural network features two distinct paths:
- The main path includes an averaging pooling layer, which aids in reducing dimensionality and smoothing outliers.
- A secondary path focuses on features representing pawns and kings. These pieces encapsulate implicit information about game phases. After processing through two layers, the results are multiplied with the outputs from the first layer of the main network path.

![Model Architecture](model.png)

The weights from the training process in the Sturddle chess engine can be exported and imported in two ways. Firstly, at compile time, the training script provides an option to export weights as C++ code snippets, suitable for modern compilers. Secondly, at runtime, weights can be exported as JSON, and the engine includes a custom UCI command for importing them. This flexibility enables efficient weight handling during engine development and deployment.

## App Overview
The Sturddle Chess App, designed for Android phones and tablets and also compatible with desktops, is an educational and interactive chess platform. Developed using Kivy and Python Chess, it caters to entry-level students and those seeking to play and study chess. The app's GitHub repository can be found at [Sturddle Chess App on GitHub](https://github.com/cristivlas/sturddle-chess-app).

## Key Features

- **Built-in Board Editor**: Users can set up and modify chess positions, aiding in learning and experimentation.
- **AI Toggle**: The app allows players to engage with the AI or opt for manual game study.
- **PGN Support**: It supports Portable Game Notation (PGN) for game import, export, and sharing, complete with rendering of annotations for in-depth analysis.
- **Voice Interaction**: Offers voice command capabilities, including offline speech recognition on desktop via OpenAI Whisper, for a hands-free experience.
- **Optional Opening Books**: Players have the option to use various opening books, assisting in the exploration of different chess strategies.
- **Encyclopedia of Chess Openings (ECO) Database**: Features the ECO database, sourced from [Lichess.org](https://github.com/lichess-org/chess-openings), providing a systematic study of chess openings.
- **Puzzle Collection**: A selection of chess puzzles from the [Lichess Puzzle Database](https://database.lichess.org/#puzzles) is included to improve tactical skills.
- **Difficulty Levels and CPU Core Utilization**: The app features 9 levels of difficulty, catering to a wide range of skill levels. On the highest level, users can increase the engine's power by allocating more CPU cores, limited only by the device's capabilities.

## Target Audience and Functionality
The Sturddle Chess App is tailored for beginners and those new to chess, providing a conducive environment for learning and improvement. Notably, the app does not include time controls, focusing instead on a relaxed and educational gameplay experience.




## Assistant Feature

### Overview
The "Assistant" feature in the Sturddle Chess App is a tool that integrates the OpenAI API's function-calling capabilities, offering users a sophisticated AI partner to assist with their chess learning and playing. It enables users to interact with the app through natural language queries and commands.

### Configuration
The feature's settings allow users to:

- **Enter the OpenAI Key**: A necessary step to access the OpenAI API's functionalities.
- **Toggle Assistant**: Switch the Assistant on or off according to the user's preference.
- **Select Model**: Choose the OpenAI model for the desired balance between speed and complexity.
- **Set Endpoint URL**: The API endpoint where the Assistant sends requests.
- **Adjust Max Retry Count**: The number of attempts the Assistant makes after a failed request.
- **Set Temperature**: Modulate the predictability or diversity of the Assistant's responses.
- **Define Initial Timeout**: The wait time for a response before the Assistant times out.

### Interactive Capabilities
The Assistant can:

- Answer queries about chess openings, like explaining variants of the Sicilian Defense, and then act on follow-up commands to play a specific variant on the board or show perspectives.
- Recommend moves, comment on the current game, or assess who is ahead in a given position. For these functionalities, the Assistant interacts with the chess engine for real-time analysis.
- Respond to requests for practice scenarios, such as "checkmate in 3" or studying specific endgames. The Assistant selects appropriate puzzles from the built-in collection to match the user's query.

### Integration and Usage
Users must have an active OpenAI subscription to access the Assistant. By entering their API key, they can leverage the Assistant to enhance their chess training. Whether it's to learn new openings, analyze ongoing games, or practice specific scenarios, the Assistant serves as an on-demand coach and companion.

The Assistant is integrated with the voice command system, utilizing a grammar framework based on the `pyparsing` library for recognizing simple spoken commands such as chess moves or opening names. This setup allows for immediate interpretation and execution of basic commands derived from speech-to-text input.

- **Simple Grammar with pyparsing**: A tool that simplifies the development of parsers to interpret voice commands.
- **Voice Commands**: Recognizes and executes basic commands related to chess gameplay.

### Fallback to OpenAI API
When local command recognition is insufficient, the system escalates unrecognized commands to the OpenAI API, provided the Assistant is enabled. This ensures that the app maintains functionality even when local parsing is unsuccessful.

### Opening Names Lookup with Fuzzy and Phonetic Matching
To handle the variability in opening name pronunciation, the app uses `rapidfuzz` for fuzzy string matching and `metaphone` for phonetic matching.

- **rapidfuzz**: Implements fast string matching to accommodate for slight variations in opening name input.
- **Phonetic Matching with metaphone**: Helps match similar-sounding words despite differences in spelling, accent, or audio quality.

### Local Processing Advantage
The decision to process chess-specific tasks locally is informed by the limitations of the ChatGPT model's understanding of complex chess opening details and its inability to visualize chess positions internally.

- **Local Chess Engine**: Utilizes the in-built chess engine for accurate and efficient chess-related processing, bypassing the limitations of cloud-based NLP in understanding detailed chess information.

These technical design choices showcase a comprehensive approach to creating an effective, efficient, and user-friendly Assistant feature within the Sturddle Chess App.

## Watch the Assistant in Action

For visual demonstrations of the Sturddle Chess App's Assistant feature, you can watch the following YouTube videos:

- [Video 1](https://youtu.be/17QLxjuZvpo)
- [Video 2](https://youtu.be/_Mna8cYUTz0)

## Assistant Feature: Interaction Workflow and Exploration of Alternatives

### Current Interaction Workflow with OpenAI API
The Assistant feature's interaction involves several steps between the user, the app, and the OpenAI API:

1. **User Input Processing**: User inputs are initially processed through the app. If not handled by the pyparsing code, they are sent to the OpenAI API, packaged with the prompt, conversation history, and a JSON schema for functions.
2. **OpenAI Processing**: The OpenAI models determine if a function call back to the app is necessary.
3. **Local App Function Execution**: The app locally executes the required function and captures the result.
4. **Result and Context Submission**: The result, along with the overall context and state information, is sent back to OpenAI.
5. **Final Rendering by OpenAI**: OpenAI renders the final output based on the new input, completing the interaction cycle.

### Exploration of Alternative Approaches: Local Vector Store
An alternative approach using a local vector store was explored to potentially reduce cloud requests:

#### Considered Technologies
- **Qdrant with Fast Embeddings** ([Qdrant GitHub Repository](https://github.com/qdrant/qdrant))
- **Vectordb over Faiss** ([Vectordb GitHub Repository](https://github.com/kagisearch/vectordb))

#### Reasons for Not Proceeding
- **Dependency Overhead**: Both solutions would introduce significant additional dependencies, complicating development and maintenance.
- **Runtime Resource Constraints**: The resource requirements of these technologies were deemed excessive for low-end Android devices, potentially impacting performance and user experience.

### Conclusion on Technical Design Choices
The exploration of these alternatives highlights the importance of balancing technical capabilities with practical constraints like device capabilities and user experience. The decision to continue with the current cloud-based approach, while keeping local processing streamlined, aligns with the goal of maintaining optimal performance on low-end devices.


